In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict


torch.set_grad_enabled(False)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
def get_x(shape= (4,3,32,224,224)):
    x_pt = torch.rand(shape) * 255
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

In [3]:
shape_of_input = [4,3,32,224,224]
x_tf, x_pt = get_x(shape_of_input)

2022-09-04 09:13:42.756057: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-04 09:13:42.756127: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gsoc): /proc/driver/nvidia/version does not exist
2022-09-04 09:13:42.756869: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

In [5]:
print("shape_of_input", shape_of_input)
print("pt")
pt_model = SwinTransformer3D_pt(**cfg,drop_rate=0.4, drop_path_rate=0.)
print("++++++++\n tf")
tf_model = SwinTransformer3D(**cfg,shape_of_input=shape_of_input, drop_rate=0.4, drop_path_rate=0.)
x_tf, x_pt = get_x()


print("\n\n\n pt")
z= pt_model(x_pt)
print("------- \n tf")
# x  = tf.keras.layers.Input(tensor=x_tf)

y = tf_model(x_tf, training= False)

print("\n loading checkpoint")
checkpoint = torch.load(f'{name}.pth')
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 
pt_model.eval()

shape_of_input [4, 3, 32, 224, 224]
pt


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


++++++++
 tf
shape_of_input:  [4, 3, 32, 224, 224]



 pt
------- 
 tf
patch embed (4, 96, 16, 56, 56)
(4, 96, 50176)

 loading checkpoint


In [6]:
pt_model.eval()


In [7]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


In [8]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [9]:
y = tf_model(x_tf, training= False)
print("------")
z= pt_model(x_pt)

y.shape, z.shape, 
y[:1,:1,:1,:1,:10], z[:1,:1,:1,:1,:10]


patch embed (4, 96, 16, 56, 56)
(4, 96, 50176)
------


(<tf.Tensor: shape=(1, 1, 1, 1, 7), dtype=float32, numpy=
 array([[[[[ 0.22170477,  0.8330549 , -0.02191872,  0.29629672,
             0.04951792,  0.05335392,  0.24042714]]]]], dtype=float32)>,
 tensor([[[[[ 0.2217,  0.8331, -0.0219,  0.2963,  0.0495,  0.0534,  0.2404]]]]]))

In [10]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-4, 1e-4)

In [11]:
print("saving model \n \n","=============================================")

tf_model.save("weights/model_tiny")

saving model 
 
patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)
patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)
patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)


2022-09-04 09:14:46.429010: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)
patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)
patch embed (None, 96, 16, 56, 56)
(None, 96, 50176)


INFO:tensorflow:Assets written to: weights/model_tiny/assets


INFO:tensorflow:Assets written to: weights/model_tiny/assets


In [12]:
tf_model_loaded = tf.keras.models.load_model('weights/model_tiny')

In [13]:
tf_model_loaded.summary()

Model: "swin_transformer3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 projection (PatchEmbed3D)   multiple                  9504      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 basic_layer (BasicLayer)    multiple                  620714    
                                                                 
 basic_layer_1 (BasicLayer)  multiple                  1523924   
                                                                 
 basic_layer_2 (BasicLayer)  multiple                  12934008  
                                                                 
 basic_layer_3 (BasicLayer)  multiple                  14604752  
                                                                 
 layer_normalization_27 (Lay  multiple          

In [14]:
y_loaded  = tf_model_loaded(x_tf)
y[:1,:1,:1,:1,:10], y_loaded[:1,:1,:1,:1,:10]

(<tf.Tensor: shape=(1, 1, 1, 1, 7), dtype=float32, numpy=
 array([[[[[ 0.22170477,  0.8330549 , -0.02191872,  0.29629672,
             0.04951792,  0.05335392,  0.24042714]]]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 1, 1, 7), dtype=float32, numpy=
 array([[[[[ 0.22170512,  0.8330553 , -0.02191865,  0.2962963 ,
             0.04951816,  0.05335341,  0.2404271 ]]]]], dtype=float32)>)

In [ ]:
np.testing.assert_allclose(y.numpy(), y_loaded.numpy(), 1e-4, 1e-4)
